In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, NodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                     eager = True,
                     create_inverse_triples = True
                     )


In [5]:
model_name = 'nodepiece_0_transductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
num_tokens = 20

In [8]:
model = NodePiece(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/7.39k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 2600
Space occupied: 10400 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_0_transductive created successfully!


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
#         callbacks="evaluation",
#         callback_kwargs=dict(
#             evaluator=valid_evaluator,
#             evaluation_triples=dataset.validation.mapped_triples,
#             prefix="validation",
#             frequency=1,
#             additional_filter_triples=dataset.training.mapped_triples,
#         ),
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 8.223759174346924
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.018875021500331473
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.01018880981030464
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.014531915655318056
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0017449556617066264
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.005135622341185808
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0034402888268232346
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0011797771482979836
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.003822663925075231
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0025012205366866077
Metric: validation.head.optimistic.hits_at_1 = 0.014396887159533073
Metric: validation.tail.optimistic.hits_at_1 = 0.0062256809338521405
Metric: validation.both.optimistic.hits_at_1 =

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 8.221463203430176
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.01889783178231861
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.01019916583158822
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.014548498806953416
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0017501133261248472
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0051374188624322414
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.003443766152486205
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0011832535318239859
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.003822952502010091
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0025031030169170385
Metric: validation.head.optimistic.hits_at_1 = 0.014396887159533073
Metric: validation.tail.optimistic.hits_at_1 = 0.0062256809338521405
Metric: validation.both.optimistic.hits_at_1 = 

...

Step: 147
Metric: loss = 0.6869449317455292
Step: 147
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04009025660396458
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.025253596706525244
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.03267192665524491
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.003914740402251482
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.008839002810418606
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.006376872304826975
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.002808165199727988
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.006801654935702905
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.004804910067715446
Metric: validation.head.optimistic.hits_at_1 = 0.02840466926070039
Metric: validation.tail.optimistic.hits_at_1 = 0.013618677042801557
Metric: validation.both.optimistic.hits_at_1 = 0

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 148
Metric: loss = 0.68000128865242
Step: 148
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.04010137892676412
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.025259532589704804
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.032680455758234465
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.003924069460481405
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.008840907365083694
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0063824886456131935
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.002816429815783376
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.006802234136026951
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.004809331975905163
Metric: validation.head.optimistic.hits_at_1 = 0.02840466926070039
Metric: validation.tail.optimistic.hits_at_1 = 0.013618677042801557
Metric: validation.both.optimistic.hits_at_1 = 0

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 149
Metric: loss = 0.6775033473968506
Step: 149
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0401020484805909
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.02525094981287478
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.03267649914673284
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.003927277401089668
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.008838160894811153
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.006382719613611698
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0028190707615664535
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00679986844075034
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.004809469601158397
Metric: validation.head.optimistic.hits_at_1 = 0.02840466926070039
Metric: validation.tail.optimistic.hits_at_1 = 0.013618677042801557
Metric: validation.both.optimistic.hits_at_1 = 0.0

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    8.223759
1    8.221463
2    8.059312
3    8.094317
4    7.946660
..        ...
144  0.685071
145  0.677867
146  0.686945
147  0.680001
148  0.677503

[149 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.066976   0.003252     0.002312
hits_at_1                     0.059724   0.000096     0.000096
hits_at_3                     0.061844   0.002601     0.002408
hits_at_5                     0.065215   0.004335     0.002794
hits_at_10                    0.067527   0.004913     0.004913
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.030376   0.009643     0.007170
hits_at_1                     0.018688   0.000000     0.000000
hits_at_3                     0.019073   0.006069     0.001445
hits_at_5                     0.021193   0.007032     0.006454
hits_at_10                    0.029573   0.013197     0.013101
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.048676   0.006447     0.004741
hits_at_1                     0.039206   0.000048     0.000048
hits_at_3                     0.040459  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.035520   0.003843     0.002746
hits_at_1                     0.028405   0.000000     0.000000
hits_at_3                     0.031128   0.002724     0.001556
hits_at_5                     0.036187   0.005058     0.003502
hits_at_10                    0.042023   0.006615     0.006615
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.026117   0.009501     0.007257
hits_at_1                     0.015175   0.000000     0.000000
hits_at_3                     0.015953   0.005447     0.002724
hits_at_5                     0.018288   0.007393     0.006615
hits_at_10                    0.028016   0.013619     0.013230
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.030818   0.006672     0.005001
hits_at_1                     0.021790   0.000000     0.000000
hits_at_3                     0.023541  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.035270   0.004688     0.003585
hits_at_1                     0.028135   0.000612     0.000612
hits_at_3                     0.031193   0.004281     0.003364
hits_at_5                     0.034862   0.005505     0.004893
hits_at_10                    0.040367   0.007034     0.006728
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.025227   0.009449     0.007360
hits_at_1                     0.013150   0.000000     0.000000
hits_at_3                     0.014985   0.005199     0.001835
hits_at_5                     0.018654   0.008257     0.007034
hits_at_10                    0.030887   0.014985     0.013761
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.030249   0.007068     0.005472
hits_at_1                     0.020642   0.000306     0.000306
hits_at_3                     0.023089  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   

                                                value  
0                                              cuda:0  
1            

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)